In [78]:
from lxml import etree
import re
import json
import pandas
from collections import Counter

In [7]:
nsmap = {"": "http://www.tei-c.org/ns/1.0"}

In [28]:
path = 'Riemann1880.xml'

In [29]:
tree = etree.parse(path).getroot()

In [87]:
#number of pages
page = tree.findall(".//pb", nsmap)
len(page)

98

## Extract plain text

In [88]:
def extract_text(element):
    if element.text:
        yield re.sub(r'\ +', ' ', element.text)
    for child in element.getchildren():
        yield from extract_text(child)
    if element.tail:
        yield re.sub(r'\ +', ' ', element.tail)

In [90]:
body = tree.findall(".//body", nsmap)

In [91]:
print(''.join(extract_text(body[0])))


 SKIZZE EINER NEUEN METHODE
 DER
 HARMONIELEHRE.
 VON
 DR. HUGO RIEMANN
 LEIPZIG,
 DRUCK UND VERLAG VON BREITKOPF UND HÄRTEL
 1880.
 HERRN ABBÉ
 DR. FRANZ LISZT
 VEREHRUNGSVOLL ZUGEEIGNET.
 Da meine »Musikalische Syntaxis«, zufolge der com¬
 VORWORT.
 Da sich dieses Schriftchen nicht an solche wen¬
 det, welche die Harmonielehre erlernen wollen, sondern
 vielmehr an die Lehrer selbst oder doch an solche,
 welche die Lehre bereits inne haben, so setzt es nicht
 allein die Kenntniss der Intervalle und Accorde, über¬
 haupt der Terminologie der Generalbasslehre, sondern
 auch der wichtigsten Thatsachen der Akustik, beson¬
 ders der Phänomene der Obertöne und des Mittönens
 voraus. Sein Zweck ist, die neuesten Fortschritte der
 Wissenschaft in der Erkenntniss der natürlichen Grund¬
 lagen der Harmonielehre für die Praxis zu verwerthen
 und die einer früheren Periode entstammende Gene¬
 ralbasslehre, welche nur in sehr beschränktem Masse
 von diesen Fortschritten Gewinn zu ziehen geeignet


## Export named entities

In [79]:
def export_named_entities(path):
    
    tree = etree.parse("Riemann1880.xml")
    nsmap = {
        None: "http://www.tei-c.org/ns/1.0",
    }
    
    body = tree.find("//body", namespaces=nsmap)
    assert body is not None
    
    #useful vs "useless" nodes
    structure = {'lb', 'hi', 'ab', 'pb', 'p', 'div', 'head'}
    useful = {'musical_concept', 'unclear', 'rs', 'work', 'date', 'complex_expression'}
    
    counter = Counter()
    
    page = 0
    named_entities = dict()
    i = 0
    
    for element in body.iterdescendants():
        
        tag = element.tag
        
        #remove nsmap
        if "}" in tag:
            tag = tag[tag.index("}")+1:]
            
        #page break
        if tag == "pb":
            page += 1
            
        #entities: either "rs" with an attribute (when default tags), or custom tag
        if tag in useful:
            named_entities[str(i)] = {}
            
            if tag == "rs":
                named_entities[str(i)]['tag'] = element.attrib.get("type")            
            else:
                named_entities[str(i)]['tag'] = tag

            named_entities[str(i)]['text'] = element.text
            named_entities[str(i)]['page'] = page
        
        counter[tag] +=1
        i += 1
    
    name = path.split('.')[0]
    
    #save in json file
    with open(f'{name}_ner_tags.json', 'w', encoding='utf-8') as file:
        json.dump(named_entities, file)
    return named_entities
    

In [80]:
export_named_entities(path)

{'8': {'tag': 'person', 'text': 'DR. HUGO RIEMANN', 'page': 1},
 '10': {'tag': 'place', 'text': 'LEIPZIG', 'page': 1},
 '12': {'tag': 'org', 'text': 'BREITKOPF UND HÄRTEL', 'page': 1},
 '14': {'tag': 'date', 'text': '1880', 'page': 1},
 '18': {'tag': 'person', 'text': 'HERRN ABBÉ', 'page': 2},
 '20': {'tag': 'person', 'text': 'D', 'page': 2},
 '27': {'tag': 'work', 'text': 'Musikalische Syntaxis', 'page': 3},
 '57': {'tag': 'work', 'text': 'Musikalische Syntaxis', 'page': 3},
 '60': {'tag': 'work', 'text': 'Neue', 'page': 3},
 '62': {'tag': 'work', 'text': 'Schule der Harmonik', 'page': 3},
 '63': {'tag': 'work', 'text': 'Musikalische Grammatik', 'page': 3},
 '103': {'tag': 'place', 'text': 'Bromberg', 'page': 4},
 '104': {'tag': 'date', 'text': '1880', 'page': 4},
 '106': {'tag': 'person', 'text': 'Dr. Hugo Riemann', 'page': 4},
 '241': {'tag': 'musical_concept', 'text': 'Consonante Accorde', 'page': 8},
 '244': {'tag': 'musical_concept', 'text': 'harmonische Bedeutung', 'page': 8},
 

In [94]:
name = path.split('.')[0]
ner_table = pandas.read_json(f'{name}_ner_tags.json', orient='index')

In [82]:
ner_table

,page,tag,text
8,1,person,DR. HUGO RIEMANN
10,1,place,LEIPZIG
12,1,org,BREITKOPF UND HÄRTEL
14,1,date,1880
18,2,person,HERRN ABBÉ
20,2,person,D
27,3,work,Musikalische Syntaxis
57,3,work,Musikalische Syntaxis
60,3,work,Neue
62,3,work,Schule der Harmonik


In [83]:
ner_table.groupby(by='tag').count()

,page,text
tag,,
complex_expression,3,3
date,4,4
musical_concept,114,112
org,1,1
person,9,9
place,3,3
unclear,62,59
work,9,9


## Check types of nodes

In [84]:
counter.most_common()

[('lb', 6908),
 ('hi', 292),
 ('ab', 149),
 ('musical_concept', 114),
 ('pb', 98),
 ('unclear', 62),
 ('p', 47),
 ('fw', 18),
 ('rs', 13),
 ('div', 9),
 ('work', 9),
 ('head', 8),
 ('date', 4),
 ('tone', 4),
 ('note', 3),
 ('complex_expression', 3),
 ('comment', 2),
 ('choice', 2),
 ('corr', 2),
 ('sic', 2),
 ('blackening', 1)]

In [86]:
unknown = set(counter.keys())

unknown.difference_update(structure)
unknown.difference_update(useful)

print(unknown)

{'note', 'choice', 'fw', 'sic', 'corr', 'tone', 'blackening', 'comment'}
